In [15]:
import torch
from torch import nn,optim
from torchvision import transforms,datasets
import matplotlib.pyplot as plt
import numpy as np
import os
from itertools import chain
import torchvision.utils as vutils
from tensorboardX import SummaryWriter

In [16]:
writer=SummaryWriter('/user1/faculty/cvpr/ujjwal/dhritimaan/GANs/datasets/log/vae')

In [17]:
def mnist():
    compose=transforms.Compose([transforms.ToTensor()])#,transforms.Normalize((0.5,),(0.5,))])
    outp_dir='/user1/faculty/cvpr/ujjwal/dhritimaan/GANs/datasets'
    return datasets.MNIST(root=outp_dir,train=True,transform=compose,download=True)

In [18]:
mnist_data=mnist()

In [19]:
cuda=torch.device('cuda:2')

In [20]:
dataloader=torch.utils.data.DataLoader(mnist_data,batch_size=32,shuffle=True)

In [21]:
class encoder(torch.nn.Module):
    def __init__(self):
        super(encoder,self).__init__()
        self.l1=nn.Sequential(nn.Linear(784,512),nn.LeakyReLU(0.2),nn.Dropout(0.3))
        self.l2=nn.Sequential(nn.Linear(512,256),nn.LeakyReLU(0.2),nn.Dropout(0.3))
        self.l3=nn.Sequential(nn.Linear(256,128),nn.LeakyReLU(0.2),nn.Dropout(0.3))
        self.l4=nn.Sequential(nn.Linear(128,64),nn.LeakyReLU(0.2),nn.Dropout(0.3))
        self.mean=nn.Sequential(nn.Linear(64,20))
        self.std=nn.Sequential(nn.Linear(64,20))
    def forward(self,x):
        x=self.l1(x)
        x=self.l2(x)
        x=self.l3(x)
        x=self.l4(x)
        m=self.mean(x)
        s=self.std(x)
        return m,s

In [22]:
class decoder(torch.nn.Module):
    def __init__(self):
        super(decoder,self).__init__()
        self.l1=nn.Sequential(nn.Linear(20,64),nn.LeakyReLU(0.2),nn.Dropout(0.3))
        self.l2=nn.Sequential(nn.Linear(64,128),nn.LeakyReLU(0.2),nn.Dropout(0.3))
        self.l3=nn.Sequential(nn.Linear(128,256),nn.LeakyReLU(0.2),nn.Dropout(0.3))
        self.l4=nn.Sequential(nn.Linear(256,512),nn.LeakyReLU(0.2),nn.Dropout(0.3))
        self.out=nn.Sequential(nn.Linear(512,784),nn.Sigmoid())
    def forward(self,x):
        x=self.l1(x)
        x=self.l2(x)
        x=self.l3(x)
        x=self.l4(x)
        x=self.out(x)
        return x

In [23]:
enc=encoder().to(cuda)
dec=decoder().to(cuda)

In [24]:
optimizer=optim.Adam(chain(enc.parameters(),dec.parameters()),lr=0.0001,betas=(0.5,0.999))

In [25]:
l=nn.BCELoss()

In [26]:
for a,b in enumerate(dataloader):
    break

In [27]:
b[0].shape

torch.Size([32, 1, 28, 28])

In [28]:
num_epochs=100
count=0
for epoch in range(num_epochs):
    for a,b in enumerate(dataloader):
        img=b[0]
        flatten_img=img.view(img.shape[0],784).to(cuda)
        
        optimizer.zero_grad()
        
        mean,logvar=enc(flatten_img)
        std=torch.exp(0.5*logvar)
        eps=torch.rand_like(std)
        sampled_pt=mean+eps*std
        decoded=dec(sampled_pt)
        
        loss_=l(decoded,flatten_img)
        
        loss_.backward()
        optimizer.step()
        print("epoch:{} loss:{}".format(epoch,loss_))
        if count%10==0:
            writer.add_image("gen_img",vutils.make_grid(decoded.view(img.shape[0],1,28,28),padding=2,
                                                        normalize=True),count)
            writer.add_image("inp_img",vutils.make_grid(flatten_img.view(img.shape[0],1,28,28),padding=2,
                                                        normalize=True),count)
        count=count+1

epoch:0 loss:0.6935150623321533
epoch:0 loss:0.6928335428237915
epoch:0 loss:0.6918355226516724
epoch:0 loss:0.6914888620376587
epoch:0 loss:0.6908679604530334
epoch:0 loss:0.6900092363357544
epoch:0 loss:0.6895307302474976
epoch:0 loss:0.6883412599563599
epoch:0 loss:0.6875898241996765
epoch:0 loss:0.6863352060317993
epoch:0 loss:0.6855958700180054
epoch:0 loss:0.6844658851623535
epoch:0 loss:0.6837568283081055
epoch:0 loss:0.6826246380805969
epoch:0 loss:0.6799331903457642
epoch:0 loss:0.6782780885696411
epoch:0 loss:0.676832914352417
epoch:0 loss:0.6739629507064819
epoch:0 loss:0.6728042364120483
epoch:0 loss:0.6686999201774597
epoch:0 loss:0.6662502884864807
epoch:0 loss:0.6635451316833496
epoch:0 loss:0.6577309370040894
epoch:0 loss:0.6511032581329346
epoch:0 loss:0.6432271003723145
epoch:0 loss:0.6381808519363403
epoch:0 loss:0.6312060952186584
epoch:0 loss:0.6189272999763489
epoch:0 loss:0.6018087267875671
epoch:0 loss:0.5831128358840942
epoch:0 loss:0.5648865103721619
epoch:0 l

epoch:0 loss:0.2508326470851898
epoch:0 loss:0.25426581501960754
epoch:0 loss:0.2580215334892273
epoch:0 loss:0.26522740721702576
epoch:0 loss:0.26558515429496765
epoch:0 loss:0.26353639364242554
epoch:0 loss:0.26989221572875977
epoch:0 loss:0.26612064242362976
epoch:0 loss:0.25588127970695496
epoch:0 loss:0.25793203711509705
epoch:0 loss:0.2547185719013214
epoch:0 loss:0.26524367928504944
epoch:0 loss:0.2687373161315918
epoch:0 loss:0.2456645965576172
epoch:0 loss:0.2495983988046646
epoch:0 loss:0.27674877643585205
epoch:0 loss:0.2687077224254608
epoch:0 loss:0.2702736258506775
epoch:0 loss:0.2894665598869324
epoch:0 loss:0.24900946021080017
epoch:0 loss:0.28616011142730713
epoch:0 loss:0.2643718421459198
epoch:0 loss:0.268459677696228
epoch:0 loss:0.2646065950393677
epoch:0 loss:0.24986255168914795
epoch:0 loss:0.2559239864349365
epoch:0 loss:0.2572736144065857
epoch:0 loss:0.24893975257873535
epoch:0 loss:0.25540590286254883
epoch:0 loss:0.2741965353488922
epoch:0 loss:0.27278530597

epoch:0 loss:0.2745664715766907
epoch:0 loss:0.2463354915380478
epoch:0 loss:0.2528347074985504
epoch:0 loss:0.28090500831604004
epoch:0 loss:0.2594808340072632
epoch:0 loss:0.258973091840744
epoch:0 loss:0.25285470485687256
epoch:0 loss:0.2831273078918457
epoch:0 loss:0.27391672134399414
epoch:0 loss:0.2748197913169861
epoch:0 loss:0.27109387516975403
epoch:0 loss:0.27756810188293457
epoch:0 loss:0.2640160024166107
epoch:0 loss:0.2688829004764557
epoch:0 loss:0.24872951209545135
epoch:0 loss:0.24527530372142792
epoch:0 loss:0.254754900932312
epoch:0 loss:0.25190022587776184
epoch:0 loss:0.25867006182670593
epoch:0 loss:0.24467957019805908
epoch:0 loss:0.2692631185054779
epoch:0 loss:0.2486005574464798
epoch:0 loss:0.24399349093437195
epoch:0 loss:0.2489064335823059
epoch:0 loss:0.2710314691066742
epoch:0 loss:0.2586499750614166
epoch:0 loss:0.2637602388858795
epoch:0 loss:0.2635526955127716
epoch:0 loss:0.25943154096603394
epoch:0 loss:0.2752496898174286
epoch:0 loss:0.264943420886993

epoch:0 loss:0.27270129323005676
epoch:0 loss:0.26727303862571716
epoch:0 loss:0.25309649109840393
epoch:0 loss:0.26029953360557556
epoch:0 loss:0.2400364726781845
epoch:0 loss:0.2508985996246338
epoch:0 loss:0.270293653011322
epoch:0 loss:0.24134916067123413
epoch:0 loss:0.2597981095314026
epoch:0 loss:0.25254717469215393
epoch:0 loss:0.2572965621948242
epoch:0 loss:0.2540610134601593
epoch:0 loss:0.26133257150650024
epoch:0 loss:0.270196795463562
epoch:0 loss:0.2594003975391388
epoch:0 loss:0.24845652282238007
epoch:0 loss:0.2560945749282837
epoch:0 loss:0.24472512304782867
epoch:0 loss:0.24600066244602203
epoch:0 loss:0.2483489215373993
epoch:0 loss:0.2421456128358841
epoch:0 loss:0.2549573481082916
epoch:0 loss:0.2589527368545532
epoch:0 loss:0.2415255904197693
epoch:0 loss:0.25999605655670166
epoch:0 loss:0.2471088320016861
epoch:0 loss:0.24027302861213684
epoch:0 loss:0.2606040835380554
epoch:0 loss:0.25107666850090027
epoch:0 loss:0.2604663670063019
epoch:0 loss:0.24884374439716

epoch:0 loss:0.26044437289237976
epoch:0 loss:0.257507860660553
epoch:0 loss:0.2390400767326355
epoch:0 loss:0.24781133234500885
epoch:0 loss:0.2331991344690323
epoch:0 loss:0.25060153007507324
epoch:0 loss:0.2569729685783386
epoch:0 loss:0.2616208493709564
epoch:0 loss:0.2431899905204773
epoch:0 loss:0.22294440865516663
epoch:0 loss:0.25042015314102173
epoch:0 loss:0.24406206607818604
epoch:0 loss:0.25182634592056274
epoch:0 loss:0.2511538863182068
epoch:0 loss:0.25492969155311584
epoch:0 loss:0.24188633263111115
epoch:0 loss:0.24595455825328827
epoch:0 loss:0.2560661733150482
epoch:0 loss:0.2321067601442337
epoch:0 loss:0.23679305613040924
epoch:0 loss:0.23023715615272522
epoch:0 loss:0.23700636625289917
epoch:0 loss:0.2581738531589508
epoch:0 loss:0.24652978777885437
epoch:0 loss:0.2417404055595398
epoch:0 loss:0.22944645583629608
epoch:0 loss:0.25306856632232666
epoch:0 loss:0.27025771141052246
epoch:0 loss:0.2206161469221115
epoch:0 loss:0.2303207963705063
epoch:0 loss:0.236681580

...

epoch:2 loss:0.21350549161434174
epoch:2 loss:0.21874530613422394
epoch:2 loss:0.21715286374092102
epoch:2 loss:0.20988218486309052
epoch:2 loss:0.21329142153263092
epoch:2 loss:0.2251976579427719
epoch:2 loss:0.23188598453998566
epoch:2 loss:0.2263890653848648
epoch:2 loss:0.23277944326400757
epoch:2 loss:0.21569834649562836
epoch:2 loss:0.2384093552827835
epoch:2 loss:0.23925597965717316
epoch:2 loss:0.21418316662311554
epoch:2 loss:0.20383204519748688
epoch:2 loss:0.21422317624092102
epoch:2 loss:0.22164656221866608
epoch:2 loss:0.2284734696149826
epoch:2 loss:0.2350667268037796
epoch:2 loss:0.2334359586238861
epoch:2 loss:0.23293836414813995
epoch:2 loss:0.21085086464881897
epoch:2 loss:0.23843298852443695
epoch:2 loss:0.2288329303264618
epoch:2 loss:0.2169819474220276
epoch:2 loss:0.23287664353847504
epoch:2 loss:0.21406449377536774
epoch:2 loss:0.195669487118721
epoch:2 loss:0.2136615812778473
epoch:2 loss:0.22516466677188873
epoch:2 loss:0.21464912593364716
epoch:2 loss:0.214048

epoch:2 loss:0.2070147693157196
epoch:2 loss:0.21555565297603607
epoch:2 loss:0.2076462358236313
epoch:2 loss:0.2108553797006607
epoch:2 loss:0.2084694355726242
epoch:2 loss:0.21605677902698517
epoch:2 loss:0.22965139150619507
epoch:2 loss:0.23539569973945618
epoch:2 loss:0.23737400770187378
epoch:2 loss:0.2245444655418396
epoch:2 loss:0.2096116542816162
epoch:2 loss:0.2165544331073761
epoch:2 loss:0.23143413662910461
epoch:2 loss:0.21893450617790222
epoch:2 loss:0.2070561647415161
epoch:2 loss:0.22801706194877625
epoch:2 loss:0.2218727022409439
epoch:2 loss:0.21699084341526031
epoch:2 loss:0.22792726755142212
epoch:2 loss:0.2346082478761673
epoch:2 loss:0.20005454123020172
epoch:2 loss:0.21321405470371246
epoch:2 loss:0.21289443969726562
epoch:2 loss:0.2211114913225174
epoch:2 loss:0.23846206068992615
epoch:2 loss:0.20708239078521729
epoch:2 loss:0.21394290030002594
epoch:2 loss:0.21561159193515778
epoch:2 loss:0.1992892175912857
epoch:2 loss:0.22366219758987427
epoch:2 loss:0.1970269

epoch:2 loss:0.2103227823972702
epoch:2 loss:0.2335612177848816
epoch:2 loss:0.21452097594738007
epoch:2 loss:0.23289275169372559
epoch:2 loss:0.23461562395095825
epoch:2 loss:0.19923946261405945
epoch:2 loss:0.20741978287696838
epoch:2 loss:0.20574620366096497
epoch:2 loss:0.19657431542873383
epoch:2 loss:0.2010343074798584
epoch:2 loss:0.21451027691364288
epoch:2 loss:0.22774790227413177
epoch:2 loss:0.21638980507850647
epoch:2 loss:0.21603195369243622
epoch:2 loss:0.2166370451450348
epoch:2 loss:0.2037183791399002
epoch:2 loss:0.2030976563692093
epoch:2 loss:0.20226633548736572
epoch:2 loss:0.21112975478172302
epoch:2 loss:0.21435119211673737
epoch:2 loss:0.22587329149246216
epoch:2 loss:0.2108292430639267
epoch:2 loss:0.20789501070976257
epoch:2 loss:0.2355208694934845
epoch:2 loss:0.220564603805542
epoch:2 loss:0.19739612936973572
epoch:2 loss:0.21691367030143738
epoch:2 loss:0.22801761329174042
epoch:2 loss:0.22272798418998718
epoch:2 loss:0.2108808308839798
epoch:2 loss:0.202038

epoch:2 loss:0.20995023846626282
epoch:2 loss:0.20039628446102142
epoch:2 loss:0.22037313878536224
epoch:2 loss:0.21140378713607788
epoch:2 loss:0.19509240984916687
epoch:2 loss:0.21049480140209198
epoch:2 loss:0.20329506695270538
epoch:2 loss:0.22752711176872253
epoch:2 loss:0.19984805583953857
epoch:2 loss:0.20967087149620056
epoch:2 loss:0.2166159301996231
epoch:2 loss:0.22813983261585236
epoch:2 loss:0.21107150614261627
epoch:2 loss:0.21088576316833496
epoch:2 loss:0.22024579346179962
epoch:2 loss:0.2114572376012802
epoch:2 loss:0.20799139142036438
epoch:2 loss:0.21694253385066986
epoch:2 loss:0.21550661325454712
epoch:2 loss:0.21529872715473175
epoch:2 loss:0.20473363995552063
epoch:2 loss:0.2101781815290451
epoch:2 loss:0.22103315591812134
epoch:2 loss:0.2223546952009201
epoch:2 loss:0.21227450668811798
epoch:2 loss:0.21047164499759674
epoch:2 loss:0.21703840792179108
epoch:2 loss:0.20585376024246216
epoch:2 loss:0.2172926366329193
epoch:2 loss:0.21167705953121185
epoch:2 loss:0.

KeyboardInterrupt: 

In [49]:
!nvidia-smi

Thu Dec 26 01:08:09 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.129      Driver Version: 410.129      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P6            On   | 00000000:17:00.0 Off |                    0 |
| N/A   39C    P0    72W /  90W |  14231MiB / 15261MiB |     93%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P6            On   | 00000000:45:00.0 Off |                    0 |
| N/A   44C    P0    72W /  90W |  14368MiB / 15261MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [35]:
decoded.shape

torch.Size([128, 784])